In [1]:
import gym_examples
import gym
import numpy as np
# from stable_baselines3 import PPO
import sb3_contrib as sb3
from sb3_contrib import MaskablePPO
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from gym.wrappers import FlattenObservation
from stable_baselines3.common.env_util import make_vec_env

In [2]:
sb3.__version__, gym.__version__

('1.7.0', '0.26.0')

# CNN ENV TESTING

In [3]:
env2 = gym.make("gym_examples/CNN-v0")

In [4]:
start_cnn = env2.reset()
start_cnn

({'layer_type': 0,
  'layer_depth': 0,
  'filter_depth': 0,
  'filter_size': 0,
  'fc_size': 0,
  'is_start': 1,
  'pool_size_and_stride': 0},
 {'current_network': [{'layer_type': 'conv',
    'layer_depth': 0,
    'filter_depth': 0,
    'filter_size': 0,
    'fc_size': 0,
    'image_size': 28,
    'pool_size': 0,
    'pool_stride': 0,
    'is_start': 1}],
  'current_image_size': 28,
  'current_layer_depth': 0,
  'current_num_fc_layers': 0})

In [5]:
env2.action_space

MultiDiscrete([3 9 5 4 5 2 4])

In [6]:
env2.get_valid_action_mask()

inside layer depth limit
enabling convolutions


(array([1., 0., 0.]),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 1., 1., 1., 1.]),
 array([0., 1., 1., 1.]),
 array([1., 0., 0., 0., 0.]),
 array([1., 0.]),
 array([1., 0., 0., 0.]))

In [7]:
# take an action that picks a conv layer, with 20 filters with a 3x3 kernel. 
action = np.array([0, 1, 2, 2, 0, 0, 0], dtype=np.int64)

new_obs, reward, terminated, _, info = env2.step(action)

current_state layer type = conv


In [8]:
new_obs

{'layer_type': 0,
 'layer_depth': 1,
 'filter_depth': 2,
 'filter_size': 2,
 'fc_size': 0,
 'is_start': 0,
 'pool_size_and_stride': 0}

In [9]:
info

{'current_network': [{'layer_type': 'conv',
   'layer_depth': 0,
   'filter_depth': 0,
   'filter_size': 0,
   'fc_size': 0,
   'image_size': 28,
   'pool_size': 0,
   'pool_stride': 0,
   'is_start': 1},
  {'layer_type': 'conv',
   'layer_depth': 1,
   'filter_depth': 20,
   'filter_size': 3,
   'fc_size': 0,
   'image_size': 28,
   'is_start': 0,
   'pool_size': 0,
   'pool_stride': 0}],
 'current_image_size': 26,
 'current_layer_depth': 1,
 'current_num_fc_layers': 0}

In [10]:
env2.get_valid_action_mask()

inside layer depth limit
enabling convolutions
enabling valid pools


(array([1., 1., 1.]),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 array([0., 1., 1., 1., 1.]),
 array([0., 1., 1., 1.]),
 array([0., 1., 1., 1., 1.]),
 array([1., 1.]),
 array([0., 1., 1., 1.]))

In [11]:
#make an fc layer with 512
action = np.array([2, 2, 2, 2, 1, 0, 1], dtype=np.int64)
new_obs, reward, terminated, _, info = env2.step(action)

info

{'current_network': [{'layer_type': 'conv',
   'layer_depth': 0,
   'filter_depth': 0,
   'filter_size': 0,
   'fc_size': 0,
   'image_size': 28,
   'pool_size': 0,
   'pool_stride': 0,
   'is_start': 1},
  {'layer_type': 'conv',
   'layer_depth': 1,
   'filter_depth': 20,
   'filter_size': 3,
   'fc_size': 0,
   'image_size': 28,
   'is_start': 0,
   'pool_size': 0,
   'pool_stride': 0},
  {'layer_type': 'fc',
   'layer_depth': 2,
   'filter_depth': 20,
   'filter_size': 3,
   'fc_size': 512,
   'image_size': 26,
   'is_start': 0,
   'pool_size': 5,
   'pool_stride': 3}],
 'current_image_size': 26,
 'current_layer_depth': 2,
 'current_num_fc_layers': 1}

In [12]:
env2.get_valid_action_mask() #Only fc layer allowed and fc layer with less than 512 nodes and this has to be terminal layer

inside layer depth limit


(array([0., 0., 1.]),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0.]),
 array([1., 0., 0., 0.]),
 array([0., 0., 1., 1., 1.]),
 array([0., 1.]),
 array([1., 0., 0., 0.]))

In [13]:
action = np.array([2, 3, 0, 0, 2, 1, 0], dtype=np.int64)
new_obs, reward, terminated, _, info = env2.step(action)

info

512


{'current_network': [{'layer_type': 'conv',
   'layer_depth': 0,
   'filter_depth': 0,
   'filter_size': 0,
   'fc_size': 0,
   'image_size': 28,
   'pool_size': 0,
   'pool_stride': 0,
   'is_start': 1},
  {'layer_type': 'conv',
   'layer_depth': 1,
   'filter_depth': 20,
   'filter_size': 3,
   'fc_size': 0,
   'image_size': 28,
   'is_start': 0,
   'pool_size': 0,
   'pool_stride': 0},
  {'layer_type': 'fc',
   'layer_depth': 2,
   'filter_depth': 20,
   'filter_size': 3,
   'fc_size': 512,
   'image_size': 26,
   'is_start': 0,
   'pool_size': 5,
   'pool_stride': 3},
  {'layer_type': 'fc',
   'layer_depth': 3,
   'filter_depth': 0,
   'filter_size': 0,
   'fc_size': 256,
   'image_size': 26,
   'is_start': 0,
   'pool_size': 0,
   'pool_stride': 0}],
 'current_image_size': 26,
 'current_layer_depth': 3,
 'current_num_fc_layers': 2}

In [14]:
reward

0.9834763266603114

In [5]:
# env = gym.make("gym_examples/StarWars-v0")

# PPO DEBUGGING

In [6]:
# start_state = env.reset()
# start_state

In [40]:
def masking_fn(env: gym.Env):
    return env.get_valid_action_mask()

In [41]:
from gym.spaces.utils import flatten_space, flatten

sample = env.observation_space.sample()
sample

OrderedDict([('c3po', array([2, 2])),
             ('r2d2', array([4, 4])),
             ('vader', array([2, 4]))])

In [61]:
myspace2 = gym.spaces.Dict(
    {
        "layer_type": gym.spaces.Discrete(4), # conv, pool, fc, softmax
        "layer_depth": gym.spaces.Discrete(8), # Current depth of network (8 layers max)
        "filter_depth": gym.spaces.Discrete(5), # Used for conv (64, 128, 256, 512) -- 0 is no filter
        "filter_size": gym.spaces.Discrete(4), # Used for conv and pool (1,3,5) -- 0 is no filter
        "fc_size": gym.spaces.Discrete(4), # Used for fc and softmax -- number of neurons in layer (512, 256, 128, 64)
        "terminal": gym.spaces.Discrete(2) # 0 if not terminal, 1 if terminal
    }
         )

mysample2 = myspace2.sample()
mysample2

OrderedDict([('fc_size', 2),
             ('filter_depth', 1),
             ('filter_size', 2),
             ('layer_depth', 1),
             ('layer_type', 0),
             ('terminal', 0)])

In [62]:
flatten(myspace2, mysample2)

array([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0], dtype=int64)

In [47]:
myspace = gym.spaces.MultiDiscrete([4, 8, 5, 4, 4, 2])
myspace

MultiDiscrete([4 8 5 4 4 2])

In [71]:
md_mask = tuple([np.ones(x,) for x in myspace.nvec])
md_mask

(array([1., 1., 1., 1.]),
 array([1., 1., 1., 1., 1., 1., 1., 1.]),
 array([1., 1., 1., 1., 1.]),
 array([1., 1., 1., 1.]),
 array([1., 1., 1., 1.]),
 array([1., 1.]))

In [58]:
mysample = myspace.sample()
mysample

array([1, 6, 4, 2, 2, 0], dtype=int64)

In [ ]:
# array([1, 6, 4, 2, 2, 0]) -> interpreted as [layer_type, layer_depth, filter_depth, filter_size, fc_size, terminal]
# i.e. [pool, 6th layer, 512 filter depth, 3 kernel size, 128 FC size, terminal]


In [59]:
flatten(myspace, mysample)

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0], dtype=int64)

In [18]:
env = FlattenObservation(ActionMasker(env, masking_fn))

In [19]:
env

<FlattenObservation<ActionMasker<TimeLimit<OrderEnforcing<PassiveEnvChecker<StarWarsEnv<gym_examples/StarWars-v0>>>>>>>

In [9]:
# env = ActionMasker(env, masking_fn)
model = MaskablePPO(MaskableActorCriticPolicy, env, verbose=1)
model.learn(total_timesteps=(100))

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
GenerateCNN(
  (layers): ModuleList(
    (0): Conv2d(1, 5, kernel_size=(3, 3), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(5, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Linear(in_features=250, out_features=100, bias=True)
    (5): Linear(in_features=100, out_features=10, bias=True)
  )
)
TRAINING
Epoch [1/5], Step [100/187], Loss: 0.0302
Epoch [2/5], Step [100/187], Loss: 0.0176
Epoch [3/5], Step [100/187], Loss: 0.0136
Epoch [4/5], Step [100/187], Loss: 0.0005
Epoch [5/5], Step [100/187], Loss: 0.0136
TESTING
Test Accuracy of the model on the 10000 test images: 0.9959
GenerateCNN(
  (layers): ModuleList(
    (0): Conv2d(1, 5, kernel_size=(3, 3), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode